In [ ]:
import random
import json
from pyspark.sql import SparkSession

In [ ]:
minio_connection = ""
nessie_connection = ""

In [ ]:
# move to functinon
try:
    minio_conn = json.loads(minio_connection)
except json.JSONDecodeError:
    with open('../variables/minio_connection.json', "r") as minio_connection_file:
        minio_conn = json.loads(minio_connection_file.read())

In [ ]:
class LazySparkSession:
    packages = [
        "io.delta:delta-spark_2.13:4.0.0",
        "org.apache.hadoop:hadoop-aws:3.4.0",
        "com.amazonaws:aws-java-sdk-bundle:1.12.787",
        "io.unitycatalog:unitycatalog-spark_2.13:0.3.0",
    ]

    def __init__(self, access_key, secret_key, endpoint):
        self._access_key = access_key
        self._secret_key = secret_key
        self._endpoint = endpoint
        

    def start(
        self,
        app_name: str = "Airflow Spark Delta Minio App",
        executor_memory: str = "1g",
        driver_memory: str = "1g",
        driver_maxresultsize: str = "1g",
        master_url: str = "local[*]",
    ):

        builder = (
            SparkSession
            .Builder()
            .appName(app_name)
            .config("spark.hadoop.fs.s3a.access.key", self._access_key)
            .config("spark.hadoop.fs.s3a.secret.key", self._secret_key)
            .config("spark.hadoop.fs.s3a.endpoint", self._endpoint)
            .config("spark.hadoop.delta.enableFastS3AListFrom", "true")
            #
            .config("spark.executor.memory", executor_memory)
            .config("spark.driver.memory", driver_memory)
            .config("spark.driver.maxResultSize", driver_maxresultsize)
            #
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
            #
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
            #
            .config("spark.jars.packages", ",".join(self.packages))
            .master(master_url)
        )

        return builder.getOrCreate()

In [ ]:
spark = LazySparkSession(
    access_key=minio_conn.get("access_key"), 
    secret_key=minio_conn.get("key"), 
    endpoint=minio_conn.get("endpoint")
).start()

In [ ]:
spark.version

In [ ]:
df = spark.createDataFrame(
    [(random.random(), random.random())], 
    schema="longitude double,latitude double"
)

In [ ]:
df.write.format("delta").mode("append").save("s3a://landing/geography/coordinates")

In [ ]:
spark.read.format("delta").load("s3a://landing/geography/coordinates").show()